In [11]:
from AntFolio import download_data, get_tickers, train_test_split, get_pct_returns, get_cum_returns, get_log_returns, transform_to_uniform, benchmark_equal_weights, portfolio_pct_returns, portfolio_cum_returns, portfolio_log_returns

ticker_df = get_tickers('nasdaq100')[:10]
data, updated_tickers_df= download_data(ticker_df, '2020-01-01', '2021-01-01')
train, test = train_test_split(data, train_start='2020-01-01', train_end='2020-12-01', test_start='2020-12-01', test_end='2021-01-01')

pct_returns = get_pct_returns(train)
cum_returns = get_cum_returns(train) 
log_returns = get_log_returns(train)
uniform = transform_to_uniform(train)

equal_weights = benchmark_equal_weights(updated_tickers_df)

por_pct_returns = portfolio_pct_returns(train, updated_tickers_df, equal_weights)
por_cum_returns = portfolio_cum_returns(train, updated_tickers_df, equal_weights)
por_log_returns = portfolio_log_returns(train, updated_tickers_df, equal_weights)

[*********************100%***********************]  10 of 10 completed


2024-05-29 00:18:23,727 - INFO - From the initial 10 tickers, 10 were successfully downloaded and retained.


In [15]:
from scipy.optimize import minimize
import numpy as np

def markowitz_optimization(train, updated_tickers_df, risk_free_rate=0.02):

    cov_matrix = train.cov()
    mean_returns = train.mean()
    num_assets = len(updated_tickers_df)

    def portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate):
        returns = np.sum(mean_returns * weights)
        volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe_ratio = (returns - risk_free_rate) / volatility
        return returns, volatility, sharpe_ratio

    def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
        return -portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate)[2]

    constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
    bounds = tuple((0, 1) for asset in range(num_assets))
    initial_guess = num_assets * [1. / num_assets]

    result = minimize(negative_sharpe_ratio, initial_guess, args=(mean_returns, cov_matrix, risk_free_rate),
                      method='SLSQP', bounds=bounds, constraints=constraints)
    
    optimal_weights = result.x

    returns, volatility, sharpe_ratio = portfolio_performance(optimal_weights, mean_returns, cov_matrix, risk_free_rate)
    
    portfolio_performance = {
        'Expected Return': returns,
        'Volatility': volatility,
        'Sharpe Ratio': sharpe_ratio
    }

    return optimal_weights, portfolio_performance

# Example usage:
optimal_weights, portfolio_performance = markowitz_optimization(pct_returns, updated_tickers_df) 
print("Optimal Weights:", optimal_weights)
#print("Portfolio Performance:", portfolio_performance)


Optimal Weights: [5.26625020e-17 7.72564730e-19 1.49490753e-16 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 7.77156117e-16 1.00000000e+00]


In [ ]:
mpt_cum_returns = portfolio_cum_returns(train, updated_tickers_df, optimal_weights)
equal_cum_returns = portfolio_cum_returns(train, updated_tickers_df, equal_weights)

